In [2]:
import sys
import os

# Add the project root to sys.path (not src directly!)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [6]:
import pandas as pd
import numpy as np
from src.recommender import ItemBasedCF
from src.evaluation import calculate_rmse

In [4]:
user_item_matrix_ibcf = pd.read_csv(r'C:\Users\PC\Desktop\collaborative-filtering-recommender\results\user_item_matrix.csv', index_col='user_id_ml')

In [5]:
ibcf_model = ItemBasedCF(min_common_users=5)
ibcf_model.fit(user_item_matrix_ibcf)

In [7]:
def predict_ratings_ibcf(model, test_df):
    predictions = []
    for _, row in test_df.iterrows():
        pred = model.predict_rating_ibcf(row['user_id_ml'], row['item_id_ml'])
        predictions.append(pred)
    return np.array(predictions)

In [8]:
test_ratings = pd.read_csv(r'C:\Users\PC\Desktop\collaborative-filtering-recommender\results\test_ratings.csv')


In [9]:
ibcf_predictions = predict_ratings_ibcf(ibcf_model, test_ratings)
test_ratings['predicted_rating_ibcf'] = ibcf_predictions

In [11]:
test_ratings.to_csv(r'C:\Users\PC\Desktop\collaborative-filtering-recommender\results\ibcf_predictions.csv', index=False)

In [14]:
def generate_topn_ibcf(user_id, n=10):
    user_ratings = user_item_matrix_ibcf.loc[user_id]
    unrated_items = user_ratings[user_ratings.isna()].index
    
    preds = []
    for item_id in unrated_items:
        # Convert item_id to integer explicitly
        item_id_int = int(item_id)
        pred = ibcf_model.predict_rating_ibcf(user_id, item_id_int)
        if not np.isnan(pred):
            preds.append((item_id_int, pred))
    
    return sorted(preds, key=lambda x: x[1], reverse=True)[:n]

In [15]:
user_id_example = 7
top10_ibcf = generate_topn_ibcf(user_id_example)
print(f"Top 10 recommendations for user {user_id_example}:")
for item_id, rating in top10_ibcf:
    print(f"Item {item_id}: Predicted rating {rating:.2f}")

Top 10 recommendations for user 7:
Item 48: Predicted rating 5.00
Item 169: Predicted rating 5.00
Item 209: Predicted rating 5.00
Item 276: Predicted rating 5.00
Item 508: Predicted rating 5.00
Item 531: Predicted rating 5.00
Item 712: Predicted rating 5.00
Item 1149: Predicted rating 5.00
Item 58: Predicted rating 4.89
Item 87: Predicted rating 4.89
